In [93]:
# Import necessary packages
import pandas as pd
import numpy as np
import datetime
from tabulate import tabulate
from tableone import TableOne
from vigipy import *
from vigipy.utils import test_dispersion

In [94]:
# Load in file
file_path = r'C:\Users\katel\OneDrive\Documents\2024-2025\CMDA 4864\cleaned_target_drugs.csv' 
df = pd.read_csv(file_path)

data_dictionary = r"faers_analysis_dataset_dictionary.xlsx"
df_dictionary = pd.read_excel(data_dictionary, sheet_name=None)
for sheet_name, drug in df_dictionary.items():
    # Perform your processing here
    print(f"Sheet name: {sheet_name}")

#Get the sheet named Psychiatric Disorder Reac Grp from df_dictionary
df_faers_data_dictionary = df_dictionary['faers_2004_2024q1_dictionary']
df_target_drugs = df_dictionary['Target Drugs']
df_psychiatric = df_dictionary['Psychiatric Disorder Reac Grp']

Sheet name: faers_2004_2024q1_dictionary
Sheet name: Target Drugs
Sheet name: Psychiatric Disorder Reac Grp


In [95]:
#Remove the warnings
pd.options.mode.chained_assignment = None

In [96]:
#Convert the fda_dt column to datetime
df['fda_dt'] = pd.to_datetime(df['fda_dt'])
df['event_dt'] = pd.to_datetime(df['event_dt'])

print(df['ps_drugname'].unique())
print(df['fda_dt'].dt.year.unique())

['singulair' 'allegra' 'zyrtec' 'claritin' 'accolate' 'zyflo' 'xyzal']
[2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017
 2018 2019 1998 2000 2020 2021 2022 2023 2024]


In [ ]:
df_reaction_test = df.assign(reaction_pt=df['reaction_pt'].str.split(r',\s*|;')).explode('reaction_pt')
#Lowercase strings and strips whitespace in the Reaction column in df_reaction_test
df_reaction_test['reaction_pt'] = df_reaction_test['reaction_pt'].str.strip().str.lower()

#Lowercase strings in the Reaction column in df_psychiatric
df_psychiatric['Reaction'] = df_psychiatric['Reaction'].str.strip().str.lower()

#Filter the data to only include the reactions in the Psychiatric Disorder Reac Grp sheet
#df_reaction_test = df_reaction_test[df_reaction_test['reaction_pt'].isin(df_psychiatric['Reaction'].unique())]

#df_reaction_test['year'] = df_reaction_test['fda_dt'].dt.year


#Makes the reaction column in df_reaction_test a category
df_reaction_test = df_reaction_test.rename(columns={'reaction_pt': 'AE'})
df_reaction_test = df_reaction_test.rename(columns={'allergy_medication': 'name'})
df_reaction_test = df_reaction_test.rename(columns={'fda_dt': 'date'})

df_reaction_counts['count'] = df_reaction_test['AE'].value_counts()
#df_reaction_counts = df_reaction_counts.to_frame()

#df_reaction_counts = df_reaction_counts.rename(columns={'reaction_pt': 'AE'})

In [80]:
print(df_reaction_counts)

                            count
AE                               
drug ineffective            14229
dyspnoea                     6930
asthma                       6452
fatigue                      6022
off label use                5411
...                           ...
urethral obstruction            1
sunct syndrome                  1
exposure via inhalation         1
application site infection      1
hemihypoaesthesia               1

[5518 rows x 1 columns]


In [118]:
keep_columns=['AE','name','date']
df_reaction=df_reaction_test[keep_columns]
df_reaction['count']=1
df_reaction=df_reaction[df_reaction['date'].dt.year<2020]

In [119]:
df_reaction.head()

,AE,name,date,count
0,dizziness,SINGULAIR,2004-01-05,1
0,feeling hot,SINGULAIR,2004-01-05,1
0,hyperhidrosis,SINGULAIR,2004-01-05,1
1,neuritis,ALLEGRA,2004-01-06,1
1,vision blurred,ALLEGRA,2004-01-06,1


In [120]:
v_data = convert(df_reaction)
results_prr=prr(v_data)
results_prr.signals.to_excel('possible_signals_prr.xlsx', index=False)

In [121]:
results_ror=ror(v_data)
results_ror.signals.to_excel('possible_signals_ror.xlsx', index=False)

In [122]:
dispersion_data = test_dispersion(v_data)
alpha = dispersion_data["alpha"] if dispersion_data["dispersion"] > 2 else 1

results_bcpnn=bcpnn(v_data, expected_method='negative-binomial', method_alpha=alpha, min_events=3)
results_bcpnn.signals.to_excel('possible_signals_bcpnn.xlsx', index=False)

c:\Users\katel\anaconda3\lib\site-packages\vigipy-0.2.1-py3.9.egg\vigipy\utils\expectations.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`


Test data

In [2]:
file_path = r"C:\Users\katel\OneDrive\Documents\2024-2025\CMDA 4864\vigipy-master\vigipy-master\test\fixtures\sample.csv" 
df = pd.read_csv(file_path)

In [3]:
# need to tranform dataframe to have columns AE, count, name, and date 



v_data = convert(df)
results = gps(v_data, min_events=5, decision_metric='rank',
              decision_thres=1, ranking_statistic='log2', minimization_method="Nelder-Mead")
results.signals.to_excel('possible_signals.xlsx', index=False)

In [4]:
results_prr=prr(v_data)
results_prr.signals.to_excel('possible_signals_prr.xlsx', index=False)

In [5]:
results_ror=ror(v_data)
results_ror.signals.to_excel('possible_signals_ror.xlsx', index=False)

In [9]:
dispersion_data = test_dispersion(v_data)
alpha = dispersion_data["alpha"] if dispersion_data["dispersion"] > 2 else 1

bcpnn(v_data, expected_method='negative-binomial', method_alpha=alpha, min_events=3)

c:\Users\katel\anaconda3\lib\site-packages\vigipy-0.2.1-py3.9.egg\vigipy\utils\expectations.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
